In [4]:
import xarray as xr
from datetime import date
from datetime import timedelta
import xesmf as xe
import numpy as np
import calendar

In [8]:
str1 = "/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y"
str2 = "_m"
curr = date(1993,1,1)
str3 = ".nc"

str4 = str1 + str(curr.year) + str2 + "0" + str(curr.month) + str3;
    
sla = xr.open_mfdataset(str4).sel(latitude = slice(0,45) , longitude = slice(45,105))

while curr <= date(1996,11,1):
    days_in_month = calendar.monthrange(curr.year, curr.month)[1]
    curr += timedelta(days = days_in_month)
    print(str4)
    if curr.month < 10:
        str4 = str1 + str(curr.year) + str2 + "0" + str(curr.month) + str3;
    else:
        str4 = str1 + str(curr.year) + str2 + str(curr.month) + str3;
    
    sla = xr.merge([sla,xr.open_mfdataset(str4).sel(latitude = slice(0,45) , longitude = slice(45,105))])

sla = sla.rename({"latitude":"lat","longitude":"lon"})

/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y1993_m01.nc
/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y1993_m02.nc
/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y1993_m03.nc
/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y1993_m04.nc
/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y1993_m05.nc
/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y1993_m06.nc
/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y1993_m07.nc
/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y1993_m08.nc
/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y1993_m09.nc
/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y19

In [10]:
sla_out = xr.Dataset({'lat': (['lat'], np.arange(-1.25, 47.25, 2.5)),'lon': (['lon'], np.arange(43.75, 107.25, 2.5))})
regridder = xe.Regridder(sla["sla"], sla_out, 'bilinear')
sla_out = regridder(sla["sla"])
sla_out

Create weight file: bilinear_180x240_20x26.nc


<xarray.DataArray 'sla' (time: 48, lat: 20, lon: 26)>
dask.array<regrid_numpy, shape=(48, 20, 26), dtype=float64, chunksize=(48, 20, 26), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1993-01-15 1993-02-15 ... 1996-12-15
  * lon      (lon) float64 43.75 46.25 48.75 51.25 ... 98.75 101.2 103.8 106.2
  * lat      (lat) float64 -1.25 1.25 3.75 6.25 8.75 ... 38.75 41.25 43.75 46.25
Attributes:
    regrid_method:  bilinear

In [12]:
precip = xr.open_mfdataset("/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/precip.mon.mean.nc")
precip = precip.sel(lat = slice(-1.25,46.25) , lon = slice(43.75,106.25))
precip = precip.sel(time = slice("1993-01-01" , "1996-12-31")).precip
precip

<xarray.DataArray 'precip' (time: 48, lat: 20, lon: 26)>
dask.array<getitem, shape=(48, 20, 26), dtype=float32, chunksize=(48, 20, 26), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -1.25 1.25 3.75 6.25 8.75 ... 38.75 41.25 43.75 46.25
  * lon      (lon) float32 43.75 46.25 48.75 51.25 ... 101.25 103.75 106.25
  * time     (time) datetime64[ns] 1993-01-01 1993-02-01 ... 1996-12-01
Attributes:
    long_name:     Average Monthly Rate of Precipitation
    valid_range:   [  0. 100.]
    units:         mm/day
    precision:     32767
    var_desc:      Precipitation
    dataset:       GPCP Version 2.3 Combined Precipitation Dataset
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Mean
    actual_range:  [ 0.     47.3274]

In [13]:
## data array to csv file for ml training

sla_out.to_dataframe().to_csv("/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/sla.csv")
precip.to_dataframe().to_csv("/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/precip.csv")